In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder


In [2]:
transform = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])

In [3]:
train_path = "xray_dataset_covid19/train"
test_path = "xray_dataset_covid19/test"

trainset = ImageFolder(root=train_path, transform=transform)
testset = ImageFolder(root=test_path, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

In [4]:
image, label = trainset[0]  
print("Image shape:", image.shape) 

Image shape: torch.Size([3, 256, 256])


In [5]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2) 
 
        self.fc1 = nn.Linear(256 * 16 * 16, 512) 
        self.fc2 = nn.Linear(512, 10)  

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = cnn()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in trainloader:
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss/len(trainloader)
    epoch_acc = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

Epoch [1/8], Loss: 1.1632, Accuracy: 54.73%
Epoch [2/8], Loss: 0.3136, Accuracy: 83.78%
Epoch [3/8], Loss: 0.2540, Accuracy: 91.89%
Epoch [4/8], Loss: 0.2094, Accuracy: 94.59%
Epoch [5/8], Loss: 0.1705, Accuracy: 93.92%
Epoch [6/8], Loss: 0.1189, Accuracy: 95.27%
Epoch [7/8], Loss: 0.0973, Accuracy: 96.62%
Epoch [8/8], Loss: 0.0721, Accuracy: 96.62%


In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in testloader:
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.9750
Precision: 0.9762
Recall: 0.9750
F1 Score: 0.9750
